In [4]:
# Let's just leave this here, for the imports later on ;)
import sympy as smp
import numpy as np

# Perlin Noise (Implementation and Description)
##### Author: Deyan Sirakov *(17 year old nerd, with a lot of free time on his hands)*

### 1. What is Perlin Noise?
So, what is Perlin Noise? If you hear this term for the first time in your life, it may sound really complicated, but believe me, it's really just some high school math and a lot of creativity. The Perlin Noise is just a gradient noise, which looks way more natural than the [https://en.wikipedia.org/wiki/White_noise](white noise) we see sometimes for example when the connection between the TV and the reciever is bad. The Perlin Noise was developed by [https://en.wikipedia.org/wiki/Ken_Perlin](Ken Perlin in) 1983 (way older than minecraft). He created it as a result of a frustration with the "machine-like" look of the CGI by that time. Perlin developed it after working on Disney's [https://en.wikipedia.org/wiki/Tron](Tron (1982)). 

###### Resources:
 Most of the information is found in the [https://en.wikipedia.org/wiki/Perlin_noise](wikipedia page)

### 2. Explanation of the implementation
Let's simply explore the algorithm in more detail

### 3. Implementation
#### (The long awaited part)

In [5]:
# Here goes the implementation of the Perlin Noise (Probably later on :Д)